In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.plot([0,1], [1,0], '*', color='green')
plt.plot([0,1],[0,1], 'x', color='blue')

x = np.array([[0,1],[1,0],[0,0],[1,1]])
y = np.array([0,0,1,1]).reshape((-1,1))
print(x.shape,y.shape)

异或(XOR)问题：按照逻辑学的解释，如果两个条件中任意一个为True，结果为True，如果两个条件都是True或都是False，结果为False。直观理解，将这四中情况在平面上展示出来，设定True=1,False=0,平面中4个点： (1:0,1),(1:1,0),(0:0,0),(0:1,1).

上一节我们的logisti回归是找到一条直线，将平面里两种不同的点分割开来，而对于异或问题中的4个点，无论如何**找不到一条直线，将这2类点完美分割开来**。解决这类问题，就需要引入深度网络。

深层网络，复合函数 g(f(x)), 让输入经过一个函数f，产出的结果再输入到另外一个函数g。如果两个函数都是线性的，得到复合函数也是线性的，仍然无法解决上面的问题，添加一些激活函数，让复合函数不再是简单的线性了。

神经网络本质是多项式回归， DNN只要足够复杂，就能拟合任何一个多项式。

反向传播算

In [ ]:
import torch 
import torch.nn as nn

# 创建一个Xor的模型
class XOrModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(XOrModel, self).__init__()
        hidden_dim = 2
        self.linear_1 = nn.Linear(input_dim, hidden_dim,bias=True)  
        self.linear_2 = nn.Linear(hidden_dim, output_dim,bias=True)  

    def forward(self, x):
        out = torch.sigmoid(self.linear_1(x))
#         out = torch.nn.functional.relu(self.linear_1(x))
        out = self.linear_2(out)
        out = torch.sigmoid(out)
        return out
    
    def get_y(self,x):
        out = torch.sigmoid(self.linear_1(x))
#         out = torch.nn.functional.relu(self.linear_1(x))
        out = self.linear_2(out)
        return out
        

learning_rate = 0.001

model = XOrModel(2, 1) #模型初始化
criterion = nn.BCELoss() #定义损失函数
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #定义最优化算法

inX = torch.as_tensor(x,dtype=torch.float32) #将numpy转成tensor
outY = torch.as_tensor(y,dtype=torch.float32)
print(inX.shape)
print(outY)

predict_Y = model(inX) #根据输入获得当前参数下的输出值
loss = criterion(predict_Y, outY) #计算误差
print('loss {}'.format(loss.item()))

for epoch in range(50):  #迭代次数
    optimizer.zero_grad() #清理模型里参数的梯度值
    predict_Y = model(inX) #根据输入获得当前参数下的输出值
    loss = criterion(predict_Y, outY) #计算误差
    loss.backward() #反向传播，计算梯度，
    optimizer.step() #更新模型参数
#     if epoch % 50 ==0:
    print('epoch {}, loss {}'.format(epoch, loss.item()))

print('epoch {}, loss {}'.format(epoch, loss.item()))

delta = 1e-7
print("state_dict:",model.state_dict())

plt.plot([0,1], [1,0], '*', color='green')
plt.plot([0,1],[0,1], 'x', color='blue')


# 怎么画出分割面，还是没搞懂，待琢磨
model_params = list(model.parameters())
print(model_params)
model_weights = model_params[0].data.numpy()
model_bias = model_params[1].data.numpy()

x_1 = np.arange(-0.1, 1.1, 0.1)
y_1 = ((x_1 * model_weights[0,0]) + model_bias[0]) / (-model_weights[0,1])
plt.plot(x_1, y_1)

x_11 = np.arange(-0.1, 1.1, 0.1)
y_11 = ((x_11 * model_weights[1,0]) + model_bias[0]) / (-model_weights[1,1])
plt.plot(x_11, y_11)

# x_2 = np.arange(-0.1, 1.1, 0.1)
# y_2 = ((x_2 * model_weights[1,0]) + model_bias[1]) / (-model_weights[1,1])
# plt.plot(x_2, y_2)
plt.show()



In [ ]:
x_11 = np.arange(-0.1, 1.1, 0.001)
tmp = np.zeros_like(x_11)
print(x_11,tmp)
print("``````")
tt = np.concatenate((x_11.reshape((-1,1)),tmp.reshape(-1,1)),axis=1)
print(tt)
y = model(torch.tensor(tt,dtype=torch.float32))
# print()
y1 = y.data.numpy().flatten()

plt.plot([0,1], [1,0], '*', color='green')
plt.plot([0,1],[0,1], 'x', color='blue')
plt.plot(x_11,y1, '.', color='blue')


y = w1*x**2 + w2*x + b ,深度网络对于多项式的拟合